# Análisis de comentarios usando BERT.

El objetivo de este cuadernillo es aterrizar y profundizar en el procesamiento del lenguaje natural (NLP) utilizando herramientas que se encuentran en el estado del arte, ver diferentes arquitecturas, etc, utilizando el mecanismo de atención que aportan los "transformers". Se utilizará [BERT](https://es.wikipedia.org/wiki/BERT_(sistema_computacional_de_comprensi%C3%B3n_de_lenguaje)) (Representación de Codificador Bidireccional de Transformadores) como modelo de propósito general de representación del lenguaje.

- [Paper de Transformers](https://arxiv.org/pdf/1706.03762.pdf)
- [Paper de BERT](https://arxiv.org/pdf/1810.04805.pdf)

Este tutorial está basado en el vídeo de ['codificandobits'](https://www.youtube.com/watch?v=mvh7DV84mr4&list=PL9E7H1rzXKFL3a7LWs70jJ2qfpqRJ1E7h&index=5&t=1478s), agregando más información en aquellas partes donde necesito más apoyo para entender los fundamentos del NLP, Transformers y BERT. En este vídeo, se entrena con la base de datos de comentarios del sitio web dedicado al análisis de películas [IMdb](https://www.imdb.com/) para, una vez entrenado, probar con otro conjunto, uno que proviene de otro sitio web diferente (comentarios que nunca ha visto la red). Este tutorial toca muchas de las partes de la manera de trabajar de las redes neuronales en cuanto a análisis y preparación de datos, parámentros de entrenamiento y conclusiones finales.

In [ ]:
!pip install transformers

De la librería transformers se usará:

- `BertModel`: Modelo preentrenado y descargado de BERT.
- `BertTokenizer`: Herramienta de tokenización de BERT para convertir de palabras a valores numéricos.
- `AdamW`: Optimizador.
- `get_linear_schedule_with_warmup`: Se utiliza para hacer descender el valor de la tasa de aprendizaje a medida que se va entrenando.

In [ ]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from textwrap import wrap

## Montaje de Google Drive y acceso al dataset

El *dataset* que se empleará es el de ["Análisis de Sentimientos" de Standford](https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) que contiene 25.000 comentarios positivos y negativos para el entrenamiento y 25.000 para test.

Montaremos la unidad de Google Drive para acceder al *dataset* que alojaremos en nuestra cuenta.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATASET_PATH = '/content/drive/My Drive/datasets/sentiment_imdb_dataset.csv'

df = pd.read_csv(DATASET_PATH)
df = df[:10000]

df.head()

## Inicialización de parámetros

Se inicializa la semilla a un valor constante para conseguir los mismos resultados que en el tutorial.

In [ ]:
# Semilla de inicialización fija.
RANDOM_SEED = 42
MAX_LEN = 200
BATCH_SIZE = 16
NCLASSES = 2

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
# Selección de GPU:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## Exploración y preparación de los datos

### Cambio de nombre a la columna "sentimiento"

In [ ]:
# Cambiamos la columna "sentimiento" por "etiqueta"
df.columns = ["review", "label"]

df.head()

### Transformación de la etiqueta a valor numérico

- `positive` --> 1
- `negative` --> 0

In [ ]:
df = pd.read_csv(DATASET_PATH) # pendiente de borrar
df = df[:10000] # pendiente de borrar

df.columns = ["review", "label"]
df['label'] = (df['label']=='positive').astype(int)
df.head()

## Construcción de una estructura válida para BERT ([CLS] y [SEP]) y tokenización

### Carga del modelo preentrenado de BERT

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Ejemplo de tokenización

In [ ]:
sample_txt = 'I really loved that movie!'
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print('Frase: ', sample_txt)
print('Tokens: ', tokens)
print('Tokens numéricos: ', token_ids)

Codificación par introducir el dato a BERT. Esto requiere introducir los caracteres especiales `[CLS]` y `[SEP]`. Para ello se utiliza el codificador `plus`. Introduciendo una frase, el tokenizador colocará automáticamente los caracteres especiales. **Retorna un diccionario con dos claves: los `input_ids` y la `atention_mask`**.


In [ ]:
encoding = tokenizer.encode_plus(
    sample_txt,
    max_length = 10,  # Longitud máxima del texto
    truncation = True,  # Corta el texto hasta cumplir el argumento anterior
    add_special_tokens = True,  # Añade estos caracteres [cls], [sep] y los caracteres vacíos
    return_token_type_ids = False,  
    pad_to_max_length = True,  # Relleno con el token de padding
    return_attention_mask = True,  # Durante el entrenamiento se presetará atención únicamente a la parte != 0
    return_tensors = 'pt'  # Convierte la frase en un contenido ya numérico para introducir a BERT.
)

In [ ]:
encoding.keys()

Podemos usar la función inversa que convierte de valores numéricos a palabras. Eso está dentro de la clase `tokenizer`.

In [ ]:
print(f"Tokenizador: {tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])}")
print(f"ids: {encoding['input_ids'][0]}")
print(f"Attention mask: {encoding['attention_mask'][0]}")

## Creación del Dataset

Dividimos el dataset de 10000 en entrenamiento y validación (8000, 2000). Pero estos 8000 datos no se pueden entregar de golpe a la red, por varios motivos: limitación de memoria RAM....aleatoriedad de las muestras. Por tanto se presenta en "trozos" o *batches*. El tamaño de este *batch* se fijará en 16 (frases). Cuando presente todos los datos (16*500 = 8000) habrá completado una época.

Para todo esto se crea una clase que hará todo esto. Heredamos de esta clase, la clase `Dataset` de Pytorch, que nos ayuda a esta partición del dataset. Esta clase será llamada por otra función: DataLoader (la que realmente lee los datos).

In [ ]:
class IMDBDataset(Dataset):

  def __init__(self, reviews, labels, tokenizer, max_len):
    self.reviews = reviews
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.reviews)

  def __getitem__(self, item):
    '''
    Esta es la función de PyTorch va leyendo para obtener esos batches de 16 
    aleatoriamente.
    '''
    review = str(self.reviews[item])  # Texto en bruto (RAW)
    label = self.labels[item]
    # Aquí, aún tenemos el texto en bruto. Vamos a tokenizarlo y agregarle 
    # los caracteres especiales
    encoding = tokenizer.encode_plus(
        review,
        max_length = self.max_len,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,  
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
    )
    return {
        'review': review,
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'label': torch.tensor(label, dtype=torch.long) # <-- lo pasamos a tensor
    }

Construimos ahora el `DataLoader` (que pytorch requiere para leer del dataset)

Convertirmos los datos (la columna review del dataset) en formato Numpy (para que pueda ser entendido por la red) y lo mismo con las etiquetas. El tokenizador ya lo tenemos calculado de antes, al igual que el tamaño máximo. Construimos el cargador de datos de pytorch con el argumento `num_workers` para que pueda procesar de manera paralela los datos.

In [ ]:
def data_loader(df, tokenizer, max_len, batch_size):

  dataset = IMDBDataset(
      reviews=df.review.to_numpy(),
      labels=df.label.to_numpy(),
      tokenizer=tokenizer,
      max_len=MAX_LEN
  )
  return DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=4)

Dividimos el dataser utilizando la función de `sklearn: train_test_split` dividiendo en 20% (0.2): 80% train y 20% de validación.

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)

train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

## Creación del modelo de acoplamiento para el ajuste del modelo de BERT a nuestro problema (*transfer learning*).

Creación de una clase para el modelo y entrenamiento.

`nn`: Neural Network

Primero se define la estructura y luego cómo se conectan las capas entre sí.

Lo que haremos es: agregar al modelo BERT (preentrenado) una red neuronal (con una única capa) con nuestro clasificador de sentimientos.

Atención con esta línea:

`self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)`

Estamos diciendo el número de neuronas de entrada y de salida. El primer argumento será la salida (el nº de neuronas) de BERT (768 neuronas de salida) y `n_classes` (2 clases, positivo o negativo).

*Nótese que podríamos haber puesto el literal `768` como el número de salidas de la capa de BERT, pero, por buenas prácticas, se extrae esa información de los parámetros del modelo ya entrenado.*

Para reducir el *overfitting* agregamos una capa adicional intermedia entre las capas de `Dropout` (descarte de conexiones de un 30% en cada época).

In [ ]:
class BERTSentimentClassifier(nn.Module):
  
  def __init__(self, n_classes):
    super(BERTSentimentClassifier, self).__init__()  # Instanciamos la clase superior
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME) # carga del modelo pre-entrenado
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes) # Capa lineal de neuronas

  def forward(self, input_ids, attention_mask): # datos de entrada del modelo de BERT
    '''
    BERT retorna dos datos (si recordamos de lo que vimos arriba) pero solo 
    nos interesa la codificación del token de clasificación, ignorando la 
    primera salida.
    '''
    _, cls_output = self.bert(
        input_ids = input_ids,
        attention_mask = attention_mask
    )
    drop_output = self.drop(cls_output)
    final_output = self.linear(drop_output)

    return final_output # aquí tenemos la clase de salida, positivo o negativo
    

In [ ]:
model = BERTSentimentClassifier(NCLASSES)
model = model.to(device)

Podemos extraer la información del modelo imprimiendolo viendo como al final de este se encuentra la conexión entre la salida de BERT y la entrada a nuestro clasificador (pasando por el dropout).

```
BERT
. . . 

(drop): Dropout(p=0.3, inplace=False)
(linear): Linear(in_features=768, out_features=2, bias=True)
```

## Entrenamiento

Esta sección en PyTorch es un poco más compleja que en Keras pero lo vemos paso a paso.

In [ ]:
EPOCH = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCH # Este será el número de veces TOTALES que se tiene que hacer iteraciones de entrenamiento
scheduler = get_linear_schedule_with_warmup( # La tasa de aprendizaje irá disminuyendo en cada Epoca
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps,
)
loss_fn = nn.CrossEntropyLoss().to(device)


## Creación del código para el entrenamiento con una iteración de prueba

In [ ]:
def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()
  losses = [] # Almacenamiento del valor del error
  correct_predictions = 0
  for batch in data_loader: # Cogerá un batch de 16
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)
    correct_predictions += torch.sum(preds==labels)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Para evitar que el gradiente crezca demasiado
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double()/n_examples, np.mean(losses) # <-- valor promedio del error

def eval_model(model, data_loader, loss_fn, device, n_examples): # evaluar modelo
  model = model.eval() # se congela el modelo (evitamos que cambien los pesos)
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds==labels)

      losses.append(loss.item())
  return correct_predictions.double()/n_examples, np.mean(losses)

In [ ]:
for epoch in range(EPOCH):
  print(f'Epoch: {epoch+1} / {EPOCH}')
  train_acc, train_loss = train_model(
      model,
      train_data_loader,
      loss_fn,
      optimizer,
      device,
      scheduler,
      len(df_train)
  )
  test_acc, test_loss = eval_model(
      model,
      test_data_loader,
      loss_fn,
      device,
      len(df_test)
  )
  print(f"Train loss: {train_loss}, accuracy: {train_acc}")
  print(f"Test loss: {test_loss}, accuracy: {test_acc}")

## Guardando el model

In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/models/setimental_analysis_using_BERT.pt')

## Evaluación del modelo

In [ ]:
def classify_film_comment(review_text):
  encoding_review = tokenizer.encode_plus(
      review_text,
      max_length=MAX_LEN,
      truncation=True,
      add_special_tokens=True,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt'
  )

  input_ids = encoding_review['input_ids'].to(device)
  attention_mask = encoding_review['attention_mask'].to(device)

  output = model(input_ids, attention_mask)

  _, pred = torch.max(output, dim=1)

  print(f'Input : {wrap(review_text)}')

  if pred:
    print(f'Output: POSITIVO')
  else:
    print(f'Output: NEGATIVO')

### Ejemplo para la evaluación

In [ ]:
review_text = "Avengers: Infinity War at least had the good taste to abstain from Jeremy Renner. No such luck in Endgame."

classify_film_comment(review_text)